# Straightening/Normalization 

In [26]:
from scipy.spatial import distance
import ipyvolume as ipv
import numpy as np
import math
import csv

### Helper methods

In [30]:
'''
getPoints - reads file with x,y,z coordinates and returns numpy arrays of each dimension
'''
def getPoints(filename):
    x = list(); y = list(); z = list()
    with open (filename, 'r') as csv_file:
        csv_reader = csv.reader (csv_file)
        for line in csv_reader:
            x.append(line[0]); y.append(line[1]); z.append(line[2])
    x = np.array(x, dtype = float); y = np.array(y, dtype = float); z = np.array(z, dtype = float)
    return (x, y, z)

'''
Input: a is the point for which you want to find the nearest neighbor 
P is the set of available points to find the nearest neighbor from 

returns p - position/index of the nearest neighbor 
d - distance between a and its nearest neighbor 
'''
def nearestNeighbor(a,P):
    d = 10000; p = -1
    for i in range(0, len(P)):
        currdist = distance.euclidean(a,P[i])
        if(currdist<d): 
            d = currdist
            p = i
    return(p,d)

'''
Swaps elements in positions a and b in a given list P
'''
def swap(P,a,b):
    temp = np.copy(P[a])
    P[a] = P[b] 
    P[b] = temp
    return P

'''
RankPC - ranks the points on the principal curve from start to end 
Input: x,y,z coordinates of points on the principal curve 
'''
def RankPC(x,y,z):
    P = np.concatenate((x[:, np.newaxis],
                       y[:, np.newaxis], 
                       z[:, np.newaxis]),
                    axis=1)
    lenP = len(P)
    
    #Finding the nearest neighbor of the first point - P0 and moving it to position P1
    p, d = nearestNeighbor(P[0],P[1:lenP])
    p+=1 #Didn't consider the 0th element while finding the position of the nearest neighbor 
    P = swap(P,1,p)

    j = 1
    while j<lenP-1:
        p0,d0 = nearestNeighbor(P[0],P[j+1:lenP]) #Finding the nearest neighbor of P0 in the remaining elements 
        p0 = p0+j+1
        pj,dj = nearestNeighbor(P[j], P[j+1:lenP]) #Finding the nearest neighbor of Pj in the remaining elements 
        pj = pj+j+1
        
        if(p0 == pj):#If they both have the same nearest neighbor
            if(d0 < dj): #If the nearest neighbor is closest to P0
                #Add the nearest neighbor to the beginning of P, and delete it from its original location 
                P = np.insert(P, 0, P[p0], axis=0); P = np.delete(P, p0+1, 0)
            else: #Else swap positions pj with j+1
                P = swap(P,j+1,pj)  
            j+=1 #incrementing by one because only one side gets an additional element 
        else: #If their nearest neighbors are different then adding both 
            P = swap(P,j+1,pj) #Swapping does not change p0
            P = np.insert(P, 0, P[p0], axis=0)
            P = np.delete(P, p0+1, 0)
            j+=2 #incrementing by two because both sides get additional elements 
    
    P = np.hsplit(P, 3)
    xr = np.concatenate(P[0], axis=0)
    yr = np.concatenate(P[1], axis=0)
    zr = np.concatenate(P[2], axis=0)
    return (xr, yr, zr)

def Display(a):
    fig = ipv.figure()
    ipv.style.use('dark')
    scatter1 = ipv.scatter(a[0][0:150],a[1][0:150],a[2][0:150],marker = 'sphere', color = 'blue')
    scatter2 = ipv.scatter(a[0][150:300],a[1][150:300],a[2][150:300],marker = 'sphere', color = 'green')
    scatter3 = ipv.scatter(a[0][300:450],a[1][300:450],a[2][300:450],marker = 'sphere', color = 'orange')
    scatter4 = ipv.scatter(a[0][450:600],a[1][450:600],a[2][450:600],marker = 'sphere', color = 'yellow')
    ipv.show()
    return()

'''
This function returns the rotation matrix that rotates unit vector a onto unit vector b 
Source: https://math.stackexchange.com/questions/180418/calculate-rotation-matrix-to-align-vector-a-to-vector-b-in-3d/897677#897677

'''
def R(a, b):
    v = np.cross(a,b)
    s = np.linalg.norm(v)
    c = np.dot(a,b)
    I = np.identity(3)
    if(s == 0): return I #Angle between vectors is zero
    vXStr = '{} {} {}; {} {} {}; {} {} {}'.format(0, -v[2], v[1], v[2], 0, -v[0], -v[1], v[0], 0)
    vx = np.matrix(vXStr)
    Rot = I + vx + np.matmul(vx, vx) * ((1 -c)/(s**2))
    return (Rot)

'''
args: Ranked PC points
returns - three vectors defining the axes for every point on the principal curve 

- Finds a vector between consecutive points on the curve
- For the first vector, defines two orthogonal vectors - this makes up the new axes for that point
- Going to the next vector, between the next set of points on the curve, and finding the rotation matrix between the two vectors
- Rotating the other two orthogonal axes by the same rotation matrix - this defines the new axes at this point
'''
def Axes(CurvePts):
    #Storing the points of the principal cuve - the points are ordered 
    x = list(); y = list(); z = list()
    x = CurvePts[0]; y = CurvePts[1]; z = CurvePts[2]

    #Finding the axes for each set of consecutive points on the curve
    xAxes = list(); yAxes= list(); zAxes = list()

    for i in range(0, len(x)-1):
        
        #Finding the current z vector 
        zA = ([(x[i+1] - x[i]), (y[i+1] - y[i]), (z[i+1] - z[i])])
        zA = (zA/np.linalg.norm(zA))
        zAxes.append(zA)
        
        if( i == 0): #Defining two orthogonal axes for the first z axis
            
            #Defining the new x axis using the fact that the dot product between two orthogonal vectors is zero
            xA = [1,1, ((1*zA[0])+(1*zA[1]))/ (-zA[2])]
            xAxes.append(xA/np.linalg.norm(xA))

            #Defining the new y axis using the fact that the cross product between two vectors is an orthogonal vector
            yA = np.cross(xAxes[0], zA)
            yAxes.append(yA/np.linalg.norm(yA))
    
        else: #Finding the rotation matrix between the current z vector and the old z vector, and rotating the old x and y vectors by the same amount to find new vectors 
            Rot = R(zA, (zAxes[i-1]))
    
            #Rotating the previous x and y axes vectors by the same rotation matrix to find the new axes
            xA = np.matmul(Rot, np.squeeze(np.asarray(xAxes[i-1])))#converting the matrix to array before using for rotation
            xAxes.append(np.squeeze(np.asarray(xA/np.linalg.norm(xA))))

            yA = np.matmul(Rot, np.squeeze(np.asarray(yAxes[i-1])))
            yAxes.append(np.squeeze(np.asarray(yA/np.linalg.norm(yA))))

    #Defining the axes for the last point to be the same as the second to last point    
    xAxes.append(np.squeeze(np.asarray(xA/np.linalg.norm(xA))))
    yAxes.append(np.squeeze(np.asarray(yA/np.linalg.norm(yA))))
    zAxes.append(zA)
    return(xAxes, yAxes, zAxes)
 
'''
Given points on a PC,
1. Finds how many points are available 
2. picks 50 points at equal intervals
'''
def pick50points(x,y,z):
    interval = math.ceil(len(x)/50)
    i = 0
    newx = list(); newy = list();newz = list()
    while(i<len(x)):
        newx.append(x[i])
        newy.append(y[i])
        newz.append(z[i])
        i+=interval
        
    newx = np.array(newx, dtype = float); newy = np.array(newy, dtype = float); newz = np.array(newz, dtype = float)
    return(newx, newy, newz)

'''
args - Points on the curve, x,y,z coordinates
returns - For each data point, point on curve to project onto
'''
def pointToProject(CurvePts, x,y,z):
    xC = list(); yC = list(); zC = list()
    xC = CurvePts[0]; yC = CurvePts[1]; zC = CurvePts[2]
    #Creating a list of the index of the point on the line that each x,y,z is closets to
    closestPointPos = 0; i=0; j=0; indexOfClosestPoints = list()
    for i in range (0, len(x)): #looping through all the x,y,z coordinates
        mindst = 10000 #Setting an upper limit on the minimum distance 
        for j in range(0, len(xC)): #For every coordinate, looping through each point on the line
            a = (x[i], y[i], z[i])
            b = (xC[j], yC[j], zC[j])
            if (distance.euclidean(a,b)) < mindst:
                mindst = (distance.euclidean(a,b))
                closestPointPos = j
        indexOfClosestPoints.append(closestPointPos)
    return (indexOfClosestPoints)

'''
args - Points on the curve, x,y,z coordinates
returns - straightened points

For every data point, projects it onto the axes of the point on the curve determined from 'pointToProject'
Find the point that comes first in the curve between the two points
Project the data point on the axes of that point to get the new points
'''
def Straighten(CurvePts, xAxis, yAxis, zAxis, x,y,z):
    xPoints = list(); yPoints = list(); zPoints = list()
    xPoints = CurvePts[0]; yPoints = CurvePts[1]; zPoints = CurvePts[2]
    
    #Finding the distance between every pair of points and storing the cumulative distances to get to that point 
    linePtsDistances = list()
    cumulativeDistance = 0 #Keeps track of cumulative distance till that point
    linePtsDistances.append(0) #Don't have to add anything for the first point 
    for i in range (0, len(xPoints)-1,1):
        a = (xPoints[i], yPoints[i], zPoints[i])
        b = (xPoints[i+1], yPoints[i+1], zPoints[i+1])
        dst = distance.euclidean(a,b)
        cumulativeDistance = cumulativeDistance + dst
        linePtsDistances.append(cumulativeDistance)

    p = pointToProject(CurvePts, x,y,z) #list of points onto which each data point should be projected onto
    sx = list(); sy = list(); sz = list()
    for i in range(0, len(x)-1):
        sx.append(np.dot([x[i], y[i], z[i]], xAxis[p[i]]))
        sy.append(np.dot([x[i], y[i], z[i]], yAxis[p[i]]))
        sz.append(np.dot([x[i], y[i], z[i]], zAxis[p[i]]) + linePtsDistances[p[i]])
    
    sx = np.array(sx, dtype = float); sy = np.array(sy, dtype = float); sz = np.array(sz, dtype = float)
    return(sx, sy, sz)

### Main

In [31]:
#Reading the principal curve points from file
PCpoints = getPoints('fitpoints.csv')

#Reading RNP coordinates from file 
RNPCoordinates = getPoints('RNPCoordinates.csv')

#Ranking PC points 
PCranked = RankPC(PCpoints[0],PCpoints[1],PCpoints[2])

#Picking 50 points of the ranked PC points 
PC50 = pick50points(PCranked[0],PCranked[1],PCranked[2])

#Defining axes for those 50 points 
axes = Axes(PC50)

#Reshaping arrays for plotting axes 
xv = np.vstack(axes[0]); yv = np.vstack(axes[1]);zv = np.vstack(axes[2])
xh = np.hsplit(xv, 3); yh = np.hsplit(yv, 3); zh = np.hsplit(zv, 3)
xx = np.concatenate(xh[0], axis=0); xy = np.concatenate(xh[1], axis=0); xz = np.concatenate(xh[2], axis=0)
yx = np.concatenate(yh[0], axis=0); yy = np.concatenate(yh[1], axis=0); yz = np.concatenate(yh[2], axis=0)
zx = np.concatenate(zh[0], axis=0); zy = np.concatenate(zh[1], axis=0); zz = np.concatenate(zh[2], axis=0)

StraightRNP = Straighten(PC50, xv, yv, zv, RNPCoordinates[0],RNPCoordinates[1],RNPCoordinates[2])

### Vizualization

In [32]:
#Viz - RNP coordinates with curve
fig = ipv.figure()
ipv.style.use('dark')
scatter = ipv.scatter(PCpoints[0],PCpoints[1],PCpoints[2],marker = 'sphere', color = 'blue')
scatter = ipv.scatter(RNPCoordinates[0],RNPCoordinates[1],RNPCoordinates[2],marker = 'sphere', color = 'green')
ipv.show()

#Viz for ranking 
#F1: Before
fig = ipv.figure()
ipv.style.use('dark')
scatter1 = ipv.scatter(PCpoints[0][0:150],PCpoints[1][0:150],PCpoints[2][0:150],marker = 'sphere', color = 'blue')
scatter2 = ipv.scatter(PCpoints[0][150:300],PCpoints[1][150:300],PCpoints[2][150:300],marker = 'sphere', color = 'green')
scatter3 = ipv.scatter(PCpoints[0][300:450],PCpoints[1][300:450],PCpoints[2][300:450],marker = 'sphere', color = 'orange')
scatter4 = ipv.scatter(PCpoints[0][450:600],PCpoints[1][450:600],PCpoints[2][450:600],marker = 'sphere', color = 'yellow')
ipv.show()
#ipv.save('StraighteningF1.html')

#F2: After 
fig = ipv.figure()
ipv.style.use('dark')
scatter1 = ipv.scatter(PCranked[0][0:150],PCranked[1][0:150],PCranked[2][0:150],marker = 'sphere', color = 'blue')
scatter2 = ipv.scatter(PCranked[0][150:300],PCranked[1][150:300],PCranked[2][150:300],marker = 'sphere', color = 'green')
scatter3 = ipv.scatter(PCranked[0][300:450],PCranked[1][300:450],PCranked[2][300:450],marker = 'sphere', color = 'orange')
scatter4 = ipv.scatter(PCranked[0][450:600],PCranked[1][450:600],PCranked[2][450:600],marker = 'sphere', color = 'yellow')
ipv.show()
#ipv.save('StraighteningF2.html')

#Viz for axes 
#Fig 1 - All PC points
fig = ipv.figure()
scatter = ipv.scatter(PCranked[0],PCranked[1],PCranked[2],marker = 'sphere', color = 'blue')
#ipv.save('StraighteningF3.html')
ipv.show()

#Fig 2 - 50 PC points
fig = ipv.figure()
scatter = ipv.scatter(PC50[0],PC50[1],PC50[2],marker = 'sphere', color = 'yellow')
#ipv.save('StraighteningF4.html')
ipv.show()

#Fig 3 - PC points with axes
fig = ipv.figure()
scatter = ipv.scatter(PC50[0],PC50[1],PC50[2],marker = 'sphere', color = 'yellow')
quiver = ipv.quiver(PC50[0],PC50[1],PC50[2],xx,xy,xz, size = 4)
quiver = ipv.quiver(PC50[0],PC50[1],PC50[2],yx,yy,yz, size = 4)
quiver = ipv.quiver(PC50[0],PC50[1],PC50[2],zx,zy,zz, size = 4)
#ipv.save('StraighteningF5.html')
ipv.show()

#Fig - Straightened 
fig = ipv.figure()
ipv.style.use('dark')
scatter = ipv.scatter(StraightRNP[0],StraightRNP[1],StraightRNP[2],marker = 'sphere', color = 'green')
ipv.show()
